# Import Data Douane To ElasticSearch

In [9]:
import pandas as pd
import os
import elasticsearch
from elasticsearch import Elasticsearch
import elasticsearch_dsl

In [10]:
df = pd.read_csv(r'National_2019_Export.txt', sep=";", header=None)
df.columns=['Flux', 'Mois', 'Annee', 'Code_CPF6','Code_A129','Code_NC8','Code_Pays','Valeur', 'Masse', 'USUP']

### Explication des champs
* Flux : le flux correspond au mouvement de la marchandise. Ce champ est valorisé à I pour
Importation et à E pour Exportation
* Mois : c'est le mois durant lequel a eu lieu le mouvement de marchandise.
* Année: c'est l'année durant laquelle a eu lieu le mouvement de marchandise.
* Code CPF6 : C'est le niveau fin de la nomenclature Classification de produits française (CPF).
La CPF est la nomenclature statistique, nationale et centrale de produits. C'est une nomenclature de
produit et de service. Elle permet aux données du commerce extérieur d'être intégrées avec d'autres
statistiques économiques telles que la production industrielle ou la consommation dans des analyses
économiques tant globales que régionales.
* Code A129 : La nomenclature A129 correspond à une nomenclature de regroupement des niveaux
de la CPF pour répondre aux besoins de la production de données de synthèse pour l'analyse
économique et la diffusion.
* Code NC8 : C'est le code de la nomenclature combinée à 8 chiffres (NC8), utilisée pour les
obligations déclaratives des opérateurs auprès de la douane, permet une connaissance détaillée du
commerce extérieur de la France : elle compte en effet un peu moins de 10 000 rubriques.
* Code Pays : C'est un code à deux caractères alphabétiques attribué à chaque pays par la
Commission des communautés européennes.
     * À l'importation, les marchandises sont relevées au compte du pays d'origine.
     * À l'exportation, les envois sont imputés au compte de la destination finale déclarée.
* Valeur : La valeur des échanges est exprimée en euros.
* Masse : Les masses sont exprimées en kilogrammes.
* USUP : Unité supplémentaire.

In [5]:
df

,Flux,Mois,Annee,Code_CPF6,Code_A129,Code_NC8,Code_Pays,Valeur,Masse,USUP
0,E,1,2019,011111,A01Z,10011100,BE,338975,1557951,0
1,E,1,2019,011111,A01Z,10011100,DE,19,24,0
2,E,1,2019,011111,A01Z,10011100,ES,1001,2567,0
3,E,1,2019,011111,A01Z,10011100,HU,233,597,0
4,E,1,2019,011111,A01Z,10011100,IT,73656,212060,0
...,...,...,...,...,...,...,...,...,...,...
2627529,E,12,2019,910220,R91Z,97060000,TH,27500,76,0
2627530,E,12,2019,910220,R91Z,97060000,TR,264000,123,0
2627531,E,12,2019,910220,R91Z,97060000,TW,41855,27,0
2627532,E,12,2019,910220,R91Z,97060000,US,4511440,50861,0


In [3]:
df1 = df[:20]
df2 = df1.T

In [4]:
dico=df2.to_dict()
dico

{0: {'Flux': 'E',
  'Mois': 1,
  'Annee': 2019,
  'Code_CPF6': '011111',
  'Code_A129': 'A01Z',
  'Code_NC8': 10011100,
  'Code_Pays': 'BE',
  'Valeur': 338975,
  'Masse': 1557951,
  'USUP': 0},
 1: {'Flux': 'E',
  'Mois': 1,
  'Annee': 2019,
  'Code_CPF6': '011111',
  'Code_A129': 'A01Z',
  'Code_NC8': 10011100,
  'Code_Pays': 'DE',
  'Valeur': 19,
  'Masse': 24,
  'USUP': 0},
 2: {'Flux': 'E',
  'Mois': 1,
  'Annee': 2019,
  'Code_CPF6': '011111',
  'Code_A129': 'A01Z',
  'Code_NC8': 10011100,
  'Code_Pays': 'ES',
  'Valeur': 1001,
  'Masse': 2567,
  'USUP': 0},
 3: {'Flux': 'E',
  'Mois': 1,
  'Annee': 2019,
  'Code_CPF6': '011111',
  'Code_A129': 'A01Z',
  'Code_NC8': 10011100,
  'Code_Pays': 'HU',
  'Valeur': 233,
  'Masse': 597,
  'USUP': 0},
 4: {'Flux': 'E',
  'Mois': 1,
  'Annee': 2019,
  'Code_CPF6': '011111',
  'Code_A129': 'A01Z',
  'Code_NC8': 10011100,
  'Code_Pays': 'IT',
  'Valeur': 73656,
  'Masse': 212060,
  'USUP': 0},
 5: {'Flux': 'E',
  'Mois': 1,
  'Annee': 2019,


In [58]:
for i in dico:
    a = json.dumps(dico[i])
    print(a)



{"Flux": "E", "Mois": 1, "Annee": 2019, "Code_CPF6": "011111", "Code_A129": "A01Z", "Code_NC8": 10011100, "Code_Pays": "BE", "Valeur": 338975, "Masse": 1557951, "USUP": 0}
{"Flux": "E", "Mois": 1, "Annee": 2019, "Code_CPF6": "011111", "Code_A129": "A01Z", "Code_NC8": 10011100, "Code_Pays": "DE", "Valeur": 19, "Masse": 24, "USUP": 0}
{"Flux": "E", "Mois": 1, "Annee": 2019, "Code_CPF6": "011111", "Code_A129": "A01Z", "Code_NC8": 10011100, "Code_Pays": "ES", "Valeur": 1001, "Masse": 2567, "USUP": 0}
{"Flux": "E", "Mois": 1, "Annee": 2019, "Code_CPF6": "011111", "Code_A129": "A01Z", "Code_NC8": 10011100, "Code_Pays": "HU", "Valeur": 233, "Masse": 597, "USUP": 0}
{"Flux": "E", "Mois": 1, "Annee": 2019, "Code_CPF6": "011111", "Code_A129": "A01Z", "Code_NC8": 10011100, "Code_Pays": "IT", "Valeur": 73656, "Masse": 212060, "USUP": 0}
{"Flux": "E", "Mois": 1, "Annee": 2019, "Code_CPF6": "011111", "Code_A129": "A01Z", "Code_NC8": 10011100, "Code_Pays": "LU", "Valeur": 190461, "Masse": 866000, "US

In [32]:
df2.to_json (r'National_2019_Export_test.json') 


In [11]:
es = elasticsearch.Elasticsearch(['http://localhost:9200/'])

In [12]:
import numpy as np
df = pd.DataFrame(np.random.randint(0,100,size=(100, 4)), columns=list('ABCD'))

INDEX="dataframe"
TYPE= "record"

def rec_to_actions(df):
    import json
    for record in df.to_dict(orient="records"):
        yield ('{ "index" : { "_index" : "%s", "_type" : "%s" }}'% (INDEX, TYPE))
        yield (json.dumps(record, default=int))

In [72]:
df

,A,B,C,D
0,40,98,65,17
1,7,99,23,28
2,90,14,52,61
3,83,5,95,4
4,33,93,88,42
...,...,...,...,...
95,32,62,46,72
96,47,21,51,19
97,30,70,45,86
98,56,12,92,26


In [13]:

e = Elasticsearch()
r = e.bulk(rec_to_actions(df))
r

POST http://localhost:9200/_bulk [status:N/A request:4.020s]
Traceback (most recent call last):
  File "C:\Users\aaras\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Users\aaras\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "C:\Users\aaras\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aaras\Anaconda3\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "C:\Users\aaras\Anaconda3\

POST http://localhost:9200/_bulk [status:N/A request:4.018s]
Traceback (most recent call last):
  File "C:\Users\aaras\Anaconda3\lib\site-packages\urllib3\connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Users\aaras\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 80, in create_connection
    raise err
  File "C:\Users\aaras\Anaconda3\lib\site-packages\urllib3\util\connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\aaras\Anaconda3\lib\site-packages\elasticsearch\connection\http_urllib3.py", line 172, in perform_request
    response = self.pool.urlopen(method, url, body, retries=Retry(False), headers=request_headers, **kw)
  File "C:\Users\aaras\Anaconda3\

ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x000001B28E95A848>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x000001B28E95A848>: Failed to establish a new connection: [WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée)

In [ ]:
r.schema